In [3]:
# Imports
from luigi.contrib.spark import PySparkTask
from luigi.parameter import IntParameter, DateSecondParameter
from luigi import LocalTarget, Task, WrapperTask
import datetime
import pandas as pd
from sklearn.metrics import classification_report
%run Classifier.ipynb

class Evaluator(Task):

    # Date for Output-File prefix
    from datetime import date, timedelta
    date = DateSecondParameter(default=datetime.datetime.now())
    
    # Method to declare the Output-File
    def output(self):
        prefix = self.date.strftime("%Y-%m-%dT%H%M%S")
        return LocalTarget("data/%s_Evaluator_out.csv" % prefix, format=UTF8)
    
    # Method to define the required Task (Preprocessor)
    def requires(self):
        return Classifier()


    # Classify the imported Data
    def run(self):
        df = pd.read_csv(self.input().path)
        
        print(df.head())
        
        # Write .csv-File
        with self.output().open("w") as out:
            df.to_csv(out, encoding="utf-8")
            


evaluator = Evaluator()
evaluator.run()


   Unnamed: 0  Unnamed: 0.1                           date  \
0           0             0  Mon, 11 Mar 2019 04:54:04 GMT   
1           1             1  Sun, 10 Mar 2019 17:45:41 GMT   
2           2             2  Sun, 10 Mar 2019 17:45:42 GMT   
3           3             3  Sun, 10 Mar 2019 17:45:43 GMT   
4           4             4  Sun, 10 Mar 2019 17:45:44 GMT   

                                                text encoding  \
0  Bestell-Hotline +41 71 228 66 77 Navigation üb...    utf-8   
1  Café und Restaurant in Seefeld, Zürich Home Ub...    UTF-8   
2  Café und Restaurant in Seefeld, Zürich Home Ub...    UTF-8   
3  Café und Restaurant in Seefeld, Zürich Home Ub...    UTF-8   
4  Café und Restaurant in Seefeld, Zürich Home Ub...    UTF-8   

                                               title  \
0    Menu - Cafe Gschwend St.Gallen | gschwind is...   
1  Menu - Bistro und italienische Spezialitäten i...   
2  Apéro/ Abendessen - apéro und italienische Küc...   
3  japanisch